In [1]:
!git clone https://github.com/sheelfshah/banditpylib.git

Cloning into 'banditpylib'...
remote: Enumerating objects: 6211, done.
remote: Counting objects: 100% (1537/1537), done.
remote: Compressing objects: 100% (918/918), done.
remote: Total 6211 (delta 1122), reused 992 (delta 608), pack-reused 4674
Receiving objects: 100% (6211/6211), 9.99 MiB | 29.06 MiB/s, done.
Resolving deltas: 100% (4511/4511), done.


In [8]:
%cd banditpylib

/content/banditpylib


In [10]:
!make install

pip install --upgrade pip
     |████████████████████████████████| 1.6MB 8.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
pip install -r requirements.txt
     |████████████████████████████████| 219 kB 9.4 MB/s 
     |████████████████████████████████| 103 kB 15.0 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 10.3 MB 15.6 MB/s 
     |████████████████████████████████| 21.6 MB 108.7 MB/s 
     |████████████████████████████████| 15.3 MB 137 kB/s 
     |████████████████████████████████| 9.9 MB 48.1 MB/s 
     |████████████████████████████████| 3.0 MB 36.5 MB/s 
     |████████████████████████████████| 1.0 MB 40.4 MB/s 
     |████████████████████████████████| 357 kB 47.5 MB/s 
     |████████████████████████████████| 280 kB 60.7 MB/s 
     |████████████████████████████████| 510 kB 59.2 MB/s 
     |████████████████████████████████| 27.4 MB 141 kB/s 
     |██████████████

In [19]:
!pip3 uninstall matplotlib
!pip3 install matplotlib==3.1.3

Found existing installation: matplotlib 3.1.3
Uninstalling matplotlib-3.1.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3-py3.7-nspkg.pth
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/matplotlib/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/tests/*
    /usr/local/lib/python3.7/dist-packages/pylab.py
Proceed (y/n)? y
  Successfully uninstalled matplotlib-3.1.3
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albu

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import tempfile
import seaborn as sns
sns.set(style="darkgrid")

from banditpylib.bandits import LinearBandit, OrdinaryBandit
from banditpylib.arms import GaussianArm
from banditpylib.protocols import SinglePlayerProtocol, trial_data_messages_to_dict
from banditpylib.learners.linear_bandit_learner import LinUCB
from banditpylib.learners.ordinary_learner import UCB

In [12]:
num_arms = 100
horizon = 2000
delta = 1/horizon
lambda_reg = 1e-3

In [13]:
features = [np.random.normal(0, size=5) for _ in range(num_arms)]
for i in range(len(features)):
  features[i] = features[i] / np.linalg.norm(features[i])

theta = np.random.normal(0, size=5)
arm_means = [feature@theta for feature in features]
arms = [GaussianArm(mean, var=1) for mean in arm_means]

In [17]:
bandit1 = LinearBandit(features, theta)
bandit2 = OrdinaryBandit(arms=arms)
learners1 = [LinUCB(num_arms, features, delta, lambda_reg)]
learners2 = [UCB(num_arms)]

In [23]:
intermediate_regrets = list(range(0, horizon+1, 50))
temp_file1 = tempfile.NamedTemporaryFile()
temp_file2 = tempfile.NamedTemporaryFile()

In [22]:
game1 = SinglePlayerProtocol(bandit1, learners1,
                             intermediate_regrets=intermediate_regrets, horizon=horizon)
game2 = SinglePlayerProtocol(bandit2, learners,
                             intermediate_regrets=intermediate_regrets, horizon=horizon)

In [24]:
game1.play(trials=200, output_filename=temp_file1.name)
game2.play(trials=200, output_filename=temp_file2.name)

In [25]:
data_df1 = trial_data_messages_to_dict(temp_file1.name)
data_df2 = trial_data_messages_to_dict(temp_file2.name)

In [39]:
import pandas as pd
data_df = pd.concat([data_df1, data_df2])

In [40]:
sns.lineplot(x='total_actions', y='regret', hue='learner', data=data_df)
plt.show()